In [5]:
import random
import itertools
import joblib
import pickle 
import os
import cvxpy

from itertools import chain, combinations
from typing import Optional
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LinearRegression

from scipy.stats import wasserstein_distance
from scipy.stats import norm
from scipy.stats import wishart
import networkx as nx
import matplotlib.pyplot as plt

from scipy.linalg import sqrtm
from scipy.special import rel_entr
from scipy.spatial.distance import jensenshannon

from src.CBN import CausalBayesianNetwork as CBN
import modularised_utils as mut
import Linear_Additive_Noise_Models as lanm
import operations as ops

import params

np.random.seed(0)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/giofelekis/opt/anaconda3/envs/erica/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/giofelekis/opt/anaconda3/envs/erica/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/giofelekis/opt/anaconda3/envs/erica/lib/python3.12/site-packages/ipykernel/kernelapp.py", lin

In [ ]:
def dual_opt_gelbrich(moments, omega, epsilon, delta):

    ll_momemts = moments['low_level']
    hl_moments = moments['high_level']

    ll_mu_hat, ll_Sigma_hat = ll_momemts[0], ll_momemts[1]
    hl_mu_hat, hl_Sigma_hat = hl_moments[0], hl_moments[1]

    HmL       = cp.Parameter(ll_mu_hat.shape)
    HSL       = cp.Parameter(ll_Sigma_hat.shape)
    HmL.value = ll_mu_hat 
    HSL.value = ll_Sigma_hat 

    HmH       = cp.Parameter(hl_mu_hat.shape)
    HSH       = cp.Parameter(hl_Sigma_hat.shape)
    HmH.value = hl_mu_hat
    HSH.value = hl_Sigma_hat

    m, n      = len(HmL), len(HmH)
    lamL        = cp.Variable((m,n), nonneg=True) 
    lamH        = cp.Variable((n,n), nonneg=True)
    
    constraints = [lamL>=0, lamH>=0]     
    
    objective = lamL*epsilon + lamH*delta + sup_mu + sup_sigma
    
    prob = cp.Problem(cp.Minimize(obj), constraints) 
    result = prob.solve()
    
    return Plans.value, result